# Imports

In [1]:
import os

# from pycaret.regression import setup, compare_models, pull, save_model, load_model
import pycaret.regression as pyc_r
import pycaret.classification as pyc_c
import pandas_profiling
import pandas as pd
from sklearn.datasets import load_iris, load_diabetes

# Min and Hourly Data

In [6]:
df_min = pd.read_csv(r'../processed_data/Combined_Data_with_Voltages.csv', index_col=0)
df_min

,PTemp_C_Max,AirTC_Avg,RH,WS_ms_Avg,WindDir,TPM1,TPM2.5,TPM10,GeffRef,TempRef,Cycle,Rainfall,Daylight,FT_Voltage1,FT_Voltage2,DN_Voltage1,DN_Voltage2
DateTime,,,,,,,,,,,,,,,,,
2022-01-10 00:00:00,14.130,14.170,70.200,0.0810,252.700,42.0,62.0,70.0,0.000559,11.0750,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-10 00:01:00,14.116,14.150,70.322,0.0742,215.844,42.0,62.0,67.0,0.001397,11.0625,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-10 00:02:00,14.102,14.130,70.444,0.0674,178.988,44.0,61.0,64.0,0.002235,11.0500,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-10 00:03:00,14.088,14.110,70.566,0.0606,142.132,45.0,64.0,68.0,0.006148,11.0250,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-10 00:04:00,14.074,14.090,70.688,0.0538,105.276,44.0,63.0,67.0,0.000559,10.9900,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-19 23:55:00,31.690,29.340,41.480,0.7820,156.400,37.0,55.0,61.0,0.036891,27.9250,7.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-19 23:56:00,31.646,29.482,41.006,0.6770,149.340,37.0,55.0,61.0,0.001537,27.9550,7.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-19 23:57:00,31.602,29.624,40.532,0.5720,142.280,33.0,49.0,53.0,0.085030,27.9800,7.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df_min.index = pd.to_datetime(df_min.index)

In [14]:
# Columns that are resampled using mean
mean_cols = [
    'PTemp_C_Max',
    'AirTC_Avg',
    'RH',
    'WS_ms_Avg',
    'TPM1',
    'TPM2.5',
    'TPM10',
    'GeffRef',
    'TempRef'
]
df_hourly = df_min.loc[:,mean_cols].resample('H').mean()
df_hourly

,PTemp_C_Max,AirTC_Avg,RH,WS_ms_Avg,TPM1,TPM2.5,TPM10,GeffRef,TempRef
DateTime,,,,,,,,,
2022-01-10 00:00:00,13.670000,13.795000,71.902500,0.112933,44.916667,64.791667,70.033333,0.006495,10.823375
2022-01-10 01:00:00,12.868333,13.155833,75.057167,0.433200,50.791667,74.925000,81.900000,0.006708,10.469500
2022-01-10 02:00:00,12.124333,11.745167,81.215333,1.485300,66.733333,100.466667,111.350000,0.006142,10.074750
2022-01-10 03:00:00,11.233167,11.026833,84.880000,1.604900,60.316667,90.125000,99.475000,0.008116,9.482000
2022-01-10 04:00:00,10.603333,10.617000,86.345000,1.674150,60.108333,93.383333,103.741667,0.009044,9.138333
...,...,...,...,...,...,...,...,...,...
2022-04-19 19:00:00,38.847000,37.026333,23.609167,0.032633,15.533333,23.616667,26.033333,0.183493,34.237000
2022-04-19 20:00:00,36.744833,35.075000,28.273500,0.142200,31.741667,46.975000,51.366667,0.025791,32.236417
2022-04-19 21:00:00,34.795000,33.594333,32.132667,0.420933,53.025000,80.208333,87.400000,0.029238,31.160583


In [15]:
sum_cols = [
    'FT_Voltage1',
    'FT_Voltage2',
    'DN_Voltage1',
    'DN_Voltage2'
]
df_hourly[sum_cols] = df_min.loc[:,sum_cols].resample('H').sum()
df_hourly

,PTemp_C_Max,AirTC_Avg,RH,WS_ms_Avg,TPM1,TPM2.5,TPM10,GeffRef,TempRef,FT_Voltage1,FT_Voltage2,DN_Voltage1,DN_Voltage2
DateTime,,,,,,,,,,,,,
2022-01-10 00:00:00,13.670000,13.795000,71.902500,0.112933,44.916667,64.791667,70.033333,0.006495,10.823375,0.0,0.0,0.00,0.00
2022-01-10 01:00:00,12.868333,13.155833,75.057167,0.433200,50.791667,74.925000,81.900000,0.006708,10.469500,0.0,0.0,0.00,0.00
2022-01-10 02:00:00,12.124333,11.745167,81.215333,1.485300,66.733333,100.466667,111.350000,0.006142,10.074750,0.0,0.0,0.00,0.00
2022-01-10 03:00:00,11.233167,11.026833,84.880000,1.604900,60.316667,90.125000,99.475000,0.008116,9.482000,0.0,0.0,0.00,0.00
2022-01-10 04:00:00,10.603333,10.617000,86.345000,1.674150,60.108333,93.383333,103.741667,0.009044,9.138333,0.0,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-19 19:00:00,38.847000,37.026333,23.609167,0.032633,15.533333,23.616667,26.033333,0.183493,34.237000,0.3,0.3,0.23,0.23
2022-04-19 20:00:00,36.744833,35.075000,28.273500,0.142200,31.741667,46.975000,51.366667,0.025791,32.236417,0.0,0.0,0.00,0.00
2022-04-19 21:00:00,34.795000,33.594333,32.132667,0.420933,53.025000,80.208333,87.400000,0.029238,31.160583,0.0,0.0,0.00,0.00


In [16]:
# Use the most frequent value of the columns ['Cycle', 'Rainfall', 'Daylight'] when resampling
df_hourly['Cycle'] = df_min['Cycle'].resample('H').agg(lambda x: x.value_counts().index[0])
df_hourly['Rainfall'] = df_min['Rainfall'].resample('H').agg(lambda x: x.value_counts().index[0])
df_hourly['Daylight'] = df_min['Daylight'].resample('H').agg(lambda x: x.value_counts().index[0])
df_hourly

,PTemp_C_Max,AirTC_Avg,RH,WS_ms_Avg,TPM1,TPM2.5,TPM10,GeffRef,TempRef,FT_Voltage1,FT_Voltage2,DN_Voltage1,DN_Voltage2,Cycle,Rainfall,Daylight
DateTime,,,,,,,,,,,,,,,,
2022-01-10 00:00:00,13.670000,13.795000,71.902500,0.112933,44.916667,64.791667,70.033333,0.006495,10.823375,0.0,0.0,0.00,0.00,3.0,0.0,0.0
2022-01-10 01:00:00,12.868333,13.155833,75.057167,0.433200,50.791667,74.925000,81.900000,0.006708,10.469500,0.0,0.0,0.00,0.00,3.0,0.0,0.0
2022-01-10 02:00:00,12.124333,11.745167,81.215333,1.485300,66.733333,100.466667,111.350000,0.006142,10.074750,0.0,0.0,0.00,0.00,3.0,0.0,0.0
2022-01-10 03:00:00,11.233167,11.026833,84.880000,1.604900,60.316667,90.125000,99.475000,0.008116,9.482000,0.0,0.0,0.00,0.00,3.0,0.0,0.0
2022-01-10 04:00:00,10.603333,10.617000,86.345000,1.674150,60.108333,93.383333,103.741667,0.009044,9.138333,0.0,0.0,0.00,0.00,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-19 19:00:00,38.847000,37.026333,23.609167,0.032633,15.533333,23.616667,26.033333,0.183493,34.237000,0.3,0.3,0.23,0.23,7.0,0.0,0.0
2022-04-19 20:00:00,36.744833,35.075000,28.273500,0.142200,31.741667,46.975000,51.366667,0.025791,32.236417,0.0,0.0,0.00,0.00,7.0,0.0,0.0
2022-04-19 21:00:00,34.795000,33.594333,32.132667,0.420933,53.025000,80.208333,87.400000,0.029238,31.160583,0.0,0.0,0.00,0.00,7.0,0.0,0.0


In [17]:
# Save hourly data
df_hourly.to_csv(r'../processed_data/Combined_Data_with_Voltages_Hourly.csv')

# Using PyCaret

In [26]:
df = df_hourly.copy()
df.drop(['Cycle', 'Rainfall'], axis=1, inplace=True)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
target_col = 0
df.drop([x for x in voltage_cols if not x == voltage_cols[target_col]], axis=1, inplace=True)
pyc_r.setup(df, target=voltage_cols[target_col])
setup_df = pyc_r.pull()
best_model = pyc_r.compare_models()
compare_df = pyc_r.pull()

,Description,Value
0,Session id,5852
1,Target,FT_Voltage1
2,Target type,Regression
3,Original data shape,"(2400, 11)"
4,Transformed data shape,"(2400, 11)"
5,Transformed train set shape,"(1680, 11)"
6,Transformed test set shape,"(720, 11)"
7,Numeric features,10
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,6.6832,163.7121,12.7211,0.9990,0.0827,0.0647,0.0460
lightgbm,Light Gradient Boosting Machine,7.2605,186.7567,13.5835,0.9989,0.3223,0.1200,0.0440
rf,Random Forest Regressor,7.5363,213.2078,14.5514,0.9987,0.0917,0.0755,0.0500
gbr,Gradient Boosting Regressor,8.7677,251.3154,15.7941,0.9985,0.4445,0.2249,0.0640
dt,Decision Tree Regressor,9.7788,387.1664,19.6346,0.9977,0.1223,0.0898,0.0170
knn,K Neighbors Regressor,11.0108,516.3512,21.9190,0.9969,0.5430,0.2274,0.0170
ada,AdaBoost Regressor,24.9241,818.1670,28.5390,0.9952,2.2289,1.4701,0.0370
ridge,Ridge Regression,22.4644,1051.2701,32.3706,0.9938,1.4682,0.9865,0.0120
br,Bayesian Ridge,22.4141,1051.2102,32.3691,0.9938,1.4636,0.9876,0.0190
lr,Linear Regression,22.4726,1051.3093,32.3714,0.9938,1.4689,0.9861,0.0150


In [27]:
df = df_hourly.copy()
df.drop(['Cycle', 'Rainfall'], axis=1, inplace=True)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
target_col = 1
df.drop([x for x in voltage_cols if not x == voltage_cols[target_col]], axis=1, inplace=True)
pyc_r.setup(df, target=voltage_cols[target_col])
setup_df = pyc_r.pull()
best_model = pyc_r.compare_models()
compare_df = pyc_r.pull()

,Description,Value
0,Session id,1529
1,Target,FT_Voltage2
2,Target type,Regression
3,Original data shape,"(2400, 11)"
4,Transformed data shape,"(2400, 11)"
5,Transformed train set shape,"(1680, 11)"
6,Transformed test set shape,"(720, 11)"
7,Numeric features,10
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,9.3327,340.2287,18.4067,0.9979,0.0807,0.0752,0.0470
rf,Random Forest Regressor,9.9393,382.6349,19.5201,0.9976,0.0907,0.0884,0.0500
lightgbm,Light Gradient Boosting Machine,10.4523,399.1307,19.9112,0.9975,0.4285,0.1440,0.0450
gbr,Gradient Boosting Regressor,10.8443,412.9699,20.2205,0.9975,0.4672,0.2667,0.0660
knn,K Neighbors Regressor,12.2190,559.9731,23.4767,0.9966,0.5529,0.2624,0.0150
dt,Decision Tree Regressor,12.6873,656.8750,25.5287,0.9959,0.1187,0.0992,0.0160
lr,Linear Regression,18.3568,704.5527,26.4317,0.9957,1.4312,1.2460,0.0190
ridge,Ridge Regression,18.3542,704.5303,26.4312,0.9957,1.4308,1.2452,0.0140
br,Bayesian Ridge,18.3261,704.1061,26.4227,0.9957,1.4262,1.2357,0.0190
lasso,Lasso Regression,18.1701,707.4288,26.4798,0.9956,1.4006,1.1620,0.0150


In [28]:
df = df_hourly.copy()
df.drop(['Cycle', 'Rainfall'], axis=1, inplace=True)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
target_col = 2
df.drop([x for x in voltage_cols if not x == voltage_cols[target_col]], axis=1, inplace=True)
pyc_r.setup(df, target=voltage_cols[target_col])
setup_df = pyc_r.pull()
best_model = pyc_r.compare_models()
compare_df = pyc_r.pull()

,Description,Value
0,Session id,8828
1,Target,DN_Voltage1
2,Target type,Regression
3,Original data shape,"(2400, 11)"
4,Transformed data shape,"(2400, 11)"
5,Transformed train set shape,"(1680, 11)"
6,Transformed test set shape,"(720, 11)"
7,Numeric features,10
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,4.7975,109.5507,10.3962,0.9994,0.0831,0.0693,0.0480
lightgbm,Light Gradient Boosting Machine,5.4892,126.8138,11.2090,0.9993,0.3361,0.1448,0.0440
gbr,Gradient Boosting Regressor,5.8366,138.3383,11.6673,0.9993,0.3081,0.1784,0.0660
rf,Random Forest Regressor,5.2211,127.8443,11.2158,0.9993,0.0914,0.0785,0.0520
dt,Decision Tree Regressor,7.1295,237.3019,15.2867,0.9987,0.1159,0.0895,0.0200
knn,K Neighbors Regressor,8.3567,389.0513,18.5286,0.9979,0.6116,0.1973,0.0190
ada,AdaBoost Regressor,19.7426,519.1815,22.7099,0.9972,2.1457,1.9394,0.0400
ridge,Ridge Regression,18.2731,817.0098,28.5219,0.9956,1.0414,0.8418,0.0160
br,Bayesian Ridge,18.2244,817.2498,28.5265,0.9956,1.0430,0.8702,0.0210
lr,Linear Regression,18.2785,817.0256,28.5221,0.9956,1.0414,0.8392,0.0210


In [29]:
df = df_hourly.copy()
df.drop(['Cycle', 'Rainfall'], axis=1, inplace=True)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
target_col = 3
df.drop([x for x in voltage_cols if not x == voltage_cols[target_col]], axis=1, inplace=True)
pyc_r.setup(df, target=voltage_cols[target_col])
setup_df = pyc_r.pull()
best_model = pyc_r.compare_models()
compare_df = pyc_r.pull()

,Description,Value
0,Session id,8270
1,Target,DN_Voltage2
2,Target type,Regression
3,Original data shape,"(2400, 11)"
4,Transformed data shape,"(2400, 11)"
5,Transformed train set shape,"(1680, 11)"
6,Transformed test set shape,"(720, 11)"
7,Numeric features,10
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,6.3020,158.8339,12.4882,0.9991,0.0818,0.0685,0.0510
lightgbm,Light Gradient Boosting Machine,7.0410,176.4995,13.1755,0.9990,0.3584,0.1172,0.0480
rf,Random Forest Regressor,6.7535,175.9268,13.2015,0.9990,0.0916,0.0768,0.0560
gbr,Gradient Boosting Regressor,7.5027,190.5836,13.7253,0.9989,0.3709,0.2508,0.0680
dt,Decision Tree Regressor,8.9847,340.3583,18.1869,0.9981,0.1234,0.0959,0.0200
knn,K Neighbors Regressor,9.7302,380.5044,19.1945,0.9978,0.6072,0.2791,0.0180
ada,AdaBoost Regressor,19.8332,537.6509,23.0985,0.9969,2.0811,1.3058,0.0420
ridge,Ridge Regression,15.7771,605.3378,24.5723,0.9966,1.0667,0.9352,0.0230
br,Bayesian Ridge,15.7105,605.3703,24.5732,0.9965,1.0651,0.9483,0.0210
lasso,Lasso Regression,15.8905,620.5477,24.8821,0.9965,1.1364,0.9211,0.0250


In [30]:
df = df_min.copy()
df.drop(['Cycle', 'Rainfall'], axis=1, inplace=True)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
target_col = 0
df.drop([x for x in voltage_cols if not x == voltage_cols[target_col]], axis=1, inplace=True)
pyc_r.setup(df, target=voltage_cols[target_col])
setup_df = pyc_r.pull()
best_model = pyc_r.compare_models()
compare_df = pyc_r.pull()

,Description,Value
0,Session id,3078
1,Target,FT_Voltage1
2,Target type,Regression
3,Original data shape,"(144000, 12)"
4,Transformed data shape,"(144000, 12)"
5,Transformed train set shape,"(100800, 12)"
6,Transformed test set shape,"(43200, 12)"
7,Numeric features,11
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.0790,0.0916,0.3020,0.9981,0.0304,0.0278,0.5070
rf,Random Forest Regressor,0.0935,0.1146,0.3376,0.9976,0.0348,0.0335,1.1380
lightgbm,Light Gradient Boosting Machine,0.1366,0.1318,0.3625,0.9972,0.0457,0.0619,0.1110
gbr,Gradient Boosting Regressor,0.1793,0.2105,0.4581,0.9956,0.0676,0.0825,2.4910
dt,Decision Tree Regressor,0.1211,0.2086,0.4556,0.9956,0.0474,0.0421,0.1230
knn,K Neighbors Regressor,0.1562,0.2292,0.4781,0.9952,0.0603,0.0755,0.0730
lr,Linear Regression,0.4392,0.5149,0.7172,0.9892,0.1626,0.2592,0.0290
ridge,Ridge Regression,0.4392,0.5149,0.7172,0.9892,0.1626,0.2592,0.0290
br,Bayesian Ridge,0.4391,0.5149,0.7172,0.9892,0.1626,0.2590,0.0270
huber,Huber Regressor,0.4073,0.5623,0.7494,0.9882,0.1146,0.1445,0.1210


In [31]:
df = df_min.copy()
df.drop(['Cycle', 'Rainfall'], axis=1, inplace=True)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
target_col = 1
df.drop([x for x in voltage_cols if not x == voltage_cols[target_col]], axis=1, inplace=True)
pyc_r.setup(df, target=voltage_cols[target_col])
setup_df = pyc_r.pull()
best_model = pyc_r.compare_models()
compare_df = pyc_r.pull()

,Description,Value
0,Session id,6184
1,Target,FT_Voltage2
2,Target type,Regression
3,Original data shape,"(144000, 12)"
4,Transformed data shape,"(144000, 12)"
5,Transformed train set shape,"(100800, 12)"
6,Transformed test set shape,"(43200, 12)"
7,Numeric features,11
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.0797,0.0986,0.3131,0.9979,0.0319,0.0296,0.6240
rf,Random Forest Regressor,0.0962,0.1192,0.3440,0.9974,0.0361,0.0363,1.1400
lightgbm,Light Gradient Boosting Machine,0.1519,0.1516,0.3886,0.9968,0.0495,0.0683,0.1200
dt,Decision Tree Regressor,0.1223,0.2084,0.4554,0.9955,0.0488,0.0453,0.1230
knn,K Neighbors Regressor,0.1661,0.2532,0.5015,0.9946,0.0629,0.0798,0.0770
gbr,Gradient Boosting Regressor,0.2075,0.2541,0.5032,0.9946,0.0695,0.0904,2.4680
lr,Linear Regression,0.3713,0.4120,0.6412,0.9912,0.1413,0.1517,0.0420
ridge,Ridge Regression,0.3713,0.4120,0.6412,0.9912,0.1413,0.1517,0.1180
br,Bayesian Ridge,0.3713,0.4120,0.6412,0.9912,0.1412,0.1517,0.0440
huber,Huber Regressor,0.3474,0.4415,0.6635,0.9905,0.1067,0.1144,0.1270


In [32]:
df = df_min.copy()
df.drop(['Cycle', 'Rainfall'], axis=1, inplace=True)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
target_col = 2
df.drop([x for x in voltage_cols if not x == voltage_cols[target_col]], axis=1, inplace=True)
pyc_r.setup(df, target=voltage_cols[target_col])
setup_df = pyc_r.pull()
best_model = pyc_r.compare_models()
compare_df = pyc_r.pull()

,Description,Value
0,Session id,2963
1,Target,DN_Voltage1
2,Target type,Regression
3,Original data shape,"(144000, 12)"
4,Transformed data shape,"(144000, 12)"
5,Transformed train set shape,"(100800, 12)"
6,Transformed test set shape,"(43200, 12)"
7,Numeric features,11
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.0812,0.1087,0.3291,0.9979,0.0335,0.0342,0.6170
lightgbm,Light Gradient Boosting Machine,0.1178,0.1275,0.3567,0.9975,0.0453,0.0666,0.1100
rf,Random Forest Regressor,0.0947,0.1361,0.3684,0.9974,0.0381,0.0410,1.2140
gbr,Gradient Boosting Regressor,0.1475,0.2161,0.4642,0.9958,0.0607,0.0855,2.4860
knn,K Neighbors Regressor,0.1384,0.2306,0.4793,0.9955,0.0594,0.0742,0.0730
dt,Decision Tree Regressor,0.1232,0.2471,0.4963,0.9952,0.0518,0.0501,0.1250
lr,Linear Regression,0.3909,0.4653,0.6818,0.9910,0.1333,0.2908,0.0400
ridge,Ridge Regression,0.3908,0.4653,0.6818,0.9910,0.1333,0.2907,0.1100
br,Bayesian Ridge,0.3908,0.4653,0.6818,0.9910,0.1332,0.2904,0.0240
huber,Huber Regressor,0.3498,0.5030,0.7088,0.9902,0.0844,0.1519,0.1180


In [33]:
df = df_min.copy()
df.drop(['Cycle', 'Rainfall'], axis=1, inplace=True)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
target_col = 3
df.drop([x for x in voltage_cols if not x == voltage_cols[target_col]], axis=1, inplace=True)
pyc_r.setup(df, target=voltage_cols[target_col])
setup_df = pyc_r.pull()
best_model = pyc_r.compare_models()
compare_df = pyc_r.pull()

,Description,Value
0,Session id,7115
1,Target,DN_Voltage2
2,Target type,Regression
3,Original data shape,"(144000, 12)"
4,Transformed data shape,"(144000, 12)"
5,Transformed train set shape,"(100800, 12)"
6,Transformed test set shape,"(43200, 12)"
7,Numeric features,11
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.0841,0.1112,0.3321,0.9978,0.0342,0.0360,0.5230
rf,Random Forest Regressor,0.0997,0.1417,0.3754,0.9972,0.0394,0.0433,1.3610
lightgbm,Light Gradient Boosting Machine,0.1313,0.1402,0.3735,0.9972,0.0486,0.0733,0.1150
gbr,Gradient Boosting Regressor,0.1633,0.2193,0.4678,0.9956,0.0637,0.0924,2.4950
knn,K Neighbors Regressor,0.1453,0.2367,0.4849,0.9953,0.0610,0.0769,0.0900
dt,Decision Tree Regressor,0.1290,0.2487,0.4983,0.9950,0.0533,0.0528,0.1210
lr,Linear Regression,0.3382,0.3874,0.6221,0.9923,0.1160,0.1881,0.1110
ridge,Ridge Regression,0.3382,0.3874,0.6221,0.9923,0.1160,0.1881,0.1120
br,Bayesian Ridge,0.3381,0.3874,0.6221,0.9923,0.1159,0.1877,0.0240
huber,Huber Regressor,0.3054,0.4103,0.6401,0.9918,0.0832,0.1276,0.1360


# Considering both exogenous and endogenous variables

In [43]:
df_hourly = pd.read_csv(r'../processed_data/Combined_Data_with_Voltages_Hourly.csv', index_col=0)
df_hourly

,PTemp_C_Max,AirTC_Avg,RH,WS_ms_Avg,TPM1,TPM2.5,TPM10,GeffRef,TempRef,FT_Voltage1,FT_Voltage2,DN_Voltage1,DN_Voltage2,Cycle,Rainfall,Daylight
DateTime,,,,,,,,,,,,,,,,
2022-01-10 00:00:00,13.670000,13.795000,71.902500,0.112933,44.916667,64.791667,70.033333,0.006495,10.823375,0.0,0.0,0.00,0.00,3.0,0.0,0.0
2022-01-10 01:00:00,12.868333,13.155833,75.057167,0.433200,50.791667,74.925000,81.900000,0.006708,10.469500,0.0,0.0,0.00,0.00,3.0,0.0,0.0
2022-01-10 02:00:00,12.124333,11.745167,81.215333,1.485300,66.733333,100.466667,111.350000,0.006142,10.074750,0.0,0.0,0.00,0.00,3.0,0.0,0.0
2022-01-10 03:00:00,11.233167,11.026833,84.880000,1.604900,60.316667,90.125000,99.475000,0.008116,9.482000,0.0,0.0,0.00,0.00,3.0,0.0,0.0
2022-01-10 04:00:00,10.603333,10.617000,86.345000,1.674150,60.108333,93.383333,103.741667,0.009044,9.138333,0.0,0.0,0.00,0.00,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-19 19:00:00,38.847000,37.026333,23.609167,0.032633,15.533333,23.616667,26.033333,0.183493,34.237000,0.3,0.3,0.23,0.23,7.0,0.0,0.0
2022-04-19 20:00:00,36.744833,35.075000,28.273500,0.142200,31.741667,46.975000,51.366667,0.025791,32.236417,0.0,0.0,0.00,0.00,7.0,0.0,0.0
2022-04-19 21:00:00,34.795000,33.594333,32.132667,0.420933,53.025000,80.208333,87.400000,0.029238,31.160583,0.0,0.0,0.00,0.00,7.0,0.0,0.0


In [44]:
# Create Column FT_Voltage1_back_1h containing the values of FT_Voltage1 shifted by 1 hour (for all Voltage columns)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
for i in range(1, 7):
    for col_name in voltage_cols:
        df_hourly[col_name + '_back_' + str(i) + 'h'] = df_hourly[col_name].shift(i)
df_hourly.columns

Index(['PTemp_C_Max', 'AirTC_Avg', 'RH', 'WS_ms_Avg', 'TPM1', 'TPM2.5',
       'TPM10', 'GeffRef', 'TempRef', 'FT_Voltage1', 'FT_Voltage2',
       'DN_Voltage1', 'DN_Voltage2', 'Cycle', 'Rainfall', 'Daylight',
       'FT_Voltage1_back_1h', 'FT_Voltage2_back_1h', 'DN_Voltage1_back_1h',
       'DN_Voltage2_back_1h', 'FT_Voltage1_back_2h', 'FT_Voltage2_back_2h',
       'DN_Voltage1_back_2h', 'DN_Voltage2_back_2h', 'FT_Voltage1_back_3h',
       'FT_Voltage2_back_3h', 'DN_Voltage1_back_3h', 'DN_Voltage2_back_3h',
       'FT_Voltage1_back_4h', 'FT_Voltage2_back_4h', 'DN_Voltage1_back_4h',
       'DN_Voltage2_back_4h', 'FT_Voltage1_back_5h', 'FT_Voltage2_back_5h',
       'DN_Voltage1_back_5h', 'DN_Voltage2_back_5h', 'FT_Voltage1_back_6h',
       'FT_Voltage2_back_6h', 'DN_Voltage1_back_6h', 'DN_Voltage2_back_6h'],
      dtype='object')

In [57]:
df = df_hourly.copy()
df.drop(['Cycle', 'Rainfall'], axis=1, inplace=True)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
target_col = 0
for col_name in df.columns:
    if not (voltage_cols[target_col] in col_name) and ('Voltage' in col_name):
        df.drop(col_name, axis=1, inplace=True)
df.dropna(inplace=True)
pyc_r.setup(df, target=voltage_cols[target_col])
setup_df = pyc_r.pull()
best_model = pyc_r.compare_models()
compare_df = pyc_r.pull()

,Description,Value
0,Session id,2931
1,Target,FT_Voltage1
2,Target type,Regression
3,Original data shape,"(2394, 17)"
4,Transformed data shape,"(2394, 17)"
5,Transformed train set shape,"(1675, 17)"
6,Transformed test set shape,"(719, 17)"
7,Numeric features,16
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,4.8955,107.1255,10.2459,0.9994,0.0485,0.0382,0.0490
lightgbm,Light Gradient Boosting Machine,5.7491,132.7363,11.4523,0.9992,0.2212,0.0841,0.0510
rf,Random Forest Regressor,5.9943,149.1453,12.1338,0.9991,0.0660,0.0540,0.0560
gbr,Gradient Boosting Regressor,6.9742,182.8212,13.4303,0.9989,0.2516,0.0748,0.0820
dt,Decision Tree Regressor,8.2991,280.7703,16.6951,0.9983,0.0796,0.0657,0.0160
lr,Linear Regression,14.6619,459.6350,21.3717,0.9973,1.3796,0.3469,0.5290
ridge,Ridge Regression,14.6586,459.6440,21.3721,0.9973,1.3792,0.3451,0.1030
br,Bayesian Ridge,14.6454,459.9133,21.3803,0.9973,1.3778,0.3300,0.0200
llar,Lasso Least Angle Regression,14.6647,473.7140,21.7081,0.9972,1.3398,0.3011,0.1060
lasso,Lasso Regression,14.7002,473.7083,21.7107,0.9972,1.3455,0.3019,0.1060


In [58]:
df = df_hourly.copy()
df.drop(['Cycle', 'Rainfall'], axis=1, inplace=True)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
target_col = 1
for col_name in df.columns:
    if not (voltage_cols[target_col] in col_name) and ('Voltage' in col_name):
        df.drop(col_name, axis=1, inplace=True)
df.dropna(inplace=True)
pyc_r.setup(df, target=voltage_cols[target_col])
setup_df = pyc_r.pull()
best_model = pyc_r.compare_models()
compare_df = pyc_r.pull()

,Description,Value
0,Session id,1576
1,Target,FT_Voltage2
2,Target type,Regression
3,Original data shape,"(2394, 17)"
4,Transformed data shape,"(2394, 17)"
5,Transformed train set shape,"(1675, 17)"
6,Transformed test set shape,"(719, 17)"
7,Numeric features,16
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,5.5441,135.2774,11.4902,0.9992,0.0517,0.0428,0.0510
lightgbm,Light Gradient Boosting Machine,6.6551,170.6386,12.9324,0.9990,0.2635,0.0836,0.0510
rf,Random Forest Regressor,6.3300,161.6796,12.6367,0.9990,0.0604,0.0531,0.0630
gbr,Gradient Boosting Regressor,7.7076,222.0891,14.7904,0.9986,0.2867,0.1018,0.0850
dt,Decision Tree Regressor,8.2533,295.9382,17.0578,0.9982,0.0860,0.0666,0.0190
en,Elastic Net,12.4483,334.7002,18.2329,0.9980,1.2724,0.4773,0.0160
llar,Lasso Least Angle Regression,12.3993,333.2911,18.1941,0.9980,1.2688,0.4642,0.0260
br,Bayesian Ridge,12.2986,330.9624,18.1301,0.9980,1.2351,0.4075,0.0200
ridge,Ridge Regression,12.3139,330.9965,18.1323,0.9980,1.2239,0.3570,0.0220
lasso,Lasso Regression,12.4271,333.2807,18.1956,0.9980,1.2717,0.4672,0.0160


In [59]:
df = df_hourly.copy()
df.drop(['Cycle', 'Rainfall'], axis=1, inplace=True)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
target_col = 2
for col_name in df.columns:
    if not (voltage_cols[target_col] in col_name) and ('Voltage' in col_name):
        df.drop(col_name, axis=1, inplace=True)
df.dropna(inplace=True)
pyc_r.setup(df, target=voltage_cols[target_col])
setup_df = pyc_r.pull()
best_model = pyc_r.compare_models()
compare_df = pyc_r.pull()

,Description,Value
0,Session id,6862
1,Target,DN_Voltage1
2,Target type,Regression
3,Original data shape,"(2394, 17)"
4,Transformed data shape,"(2394, 17)"
5,Transformed train set shape,"(1675, 17)"
6,Transformed test set shape,"(719, 17)"
7,Numeric features,16
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,4.1687,84.6491,9.1202,0.9995,0.0551,0.0425,0.0520
lightgbm,Light Gradient Boosting Machine,4.7279,92.8011,9.5535,0.9995,0.2106,0.0789,0.0540
rf,Random Forest Regressor,5.0804,119.3676,10.8423,0.9994,0.0652,0.0536,0.0600
gbr,Gradient Boosting Regressor,5.5336,121.7250,10.9543,0.9993,0.2302,0.0853,0.0850
dt,Decision Tree Regressor,6.8333,215.1468,14.5037,0.9988,0.1003,0.0730,0.0240
ridge,Ridge Regression,14.1034,420.6804,20.4611,0.9977,1.3458,0.4623,0.0240
llar,Lasso Least Angle Regression,14.1895,432.0531,20.7292,0.9977,1.2919,0.3467,0.0220
br,Bayesian Ridge,14.1085,420.8990,20.4656,0.9977,1.3431,0.4545,0.0190
lasso,Lasso Regression,14.2215,431.8677,20.7242,0.9977,1.2964,0.3447,0.0250
lr,Linear Regression,14.1052,420.7088,20.4619,0.9977,1.3469,0.4644,0.0240


In [60]:
df = df_hourly.copy()
df.drop(['Cycle', 'Rainfall'], axis=1, inplace=True)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
target_col = 3
for col_name in df.columns:
    if not (voltage_cols[target_col] in col_name) and ('Voltage' in col_name):
        df.drop(col_name, axis=1, inplace=True)
df.dropna(inplace=True)
pyc_r.setup(df, target=voltage_cols[target_col])
setup_df = pyc_r.pull()
best_model = pyc_r.compare_models()
compare_df = pyc_r.pull()

,Description,Value
0,Session id,958
1,Target,DN_Voltage2
2,Target type,Regression
3,Original data shape,"(2394, 17)"
4,Transformed data shape,"(2394, 17)"
5,Transformed train set shape,"(1675, 17)"
6,Transformed test set shape,"(719, 17)"
7,Numeric features,16
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,5.0620,101.1464,9.9709,0.9994,0.0616,0.0487,0.0580
lightgbm,Light Gradient Boosting Machine,5.9409,122.8331,11.0331,0.9993,0.2659,0.0985,0.0570
rf,Random Forest Regressor,5.6741,127.5797,11.1750,0.9993,0.0744,0.0603,0.0610
gbr,Gradient Boosting Regressor,6.5904,153.2823,12.3111,0.9991,0.2257,0.1214,0.0850
dt,Decision Tree Regressor,7.8595,252.7530,15.7524,0.9986,0.1105,0.0788,0.0260
ridge,Ridge Regression,12.3704,327.1003,18.0606,0.9981,1.2135,0.4677,0.0210
llar,Lasso Least Angle Regression,12.6630,338.3570,18.3738,0.9981,1.2575,0.6579,0.0260
br,Bayesian Ridge,12.3691,327.5176,18.0737,0.9981,1.2135,0.5147,0.0270
lasso,Lasso Regression,12.6849,338.4255,18.3760,0.9981,1.2624,0.6646,0.0230
lr,Linear Regression,12.3713,327.1013,18.0606,0.9981,1.2136,0.4657,0.0250


In [62]:
df_min = pd.read_csv(r'../processed_data/Combined_Data_with_Voltages.csv', index_col=0)
# Create Column FT_Voltage1_back_1min containing the values of FT_Voltage1 shifted by 1 min (for all Voltage columns)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
num_hours = 1
for i in range(1, num_hours * 60 + 1):
    for col_name in voltage_cols:
        df_min[col_name + '_back_' + str(i) + 'min'] = df_min[col_name].shift(i)
df_min.columns

Index(['PTemp_C_Max', 'AirTC_Avg', 'RH', 'WS_ms_Avg', 'WindDir', 'TPM1',
       'TPM2.5', 'TPM10', 'GeffRef', 'TempRef',
       ...
       'DN_Voltage1_back_58min', 'DN_Voltage2_back_58min',
       'FT_Voltage1_back_59min', 'FT_Voltage2_back_59min',
       'DN_Voltage1_back_59min', 'DN_Voltage2_back_59min',
       'FT_Voltage1_back_60min', 'FT_Voltage2_back_60min',
       'DN_Voltage1_back_60min', 'DN_Voltage2_back_60min'],
      dtype='object', length=257)

In [63]:
df = df_min.copy()
df.drop(['Cycle', 'Rainfall'], axis=1, inplace=True)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
target_col = 0
for col_name in df.columns:
    if not (voltage_cols[target_col] in col_name) and ('Voltage' in col_name):
        df.drop(col_name, axis=1, inplace=True)
df.dropna(inplace=True)
pyc_r.setup(df, target=voltage_cols[target_col])
setup_df = pyc_r.pull()
best_model = pyc_r.compare_models()
compare_df = pyc_r.pull()

,Description,Value
0,Session id,5039
1,Target,FT_Voltage1
2,Target type,Regression
3,Original data shape,"(143940, 72)"
4,Transformed data shape,"(143940, 72)"
5,Transformed train set shape,"(100758, 72)"
6,Transformed test set shape,"(43182, 72)"
7,Numeric features,71
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
omp,Orthogonal Matching Pursuit,0.0585,0.0644,0.2521,0.9987,0.0242,0.0154,0.2080
rf,Random Forest Regressor,0.0494,0.0646,0.2532,0.9987,0.0225,0.0120,5.8040
lightgbm,Light Gradient Boosting Machine,0.0540,0.0626,0.2491,0.9987,0.0226,0.0164,0.4740
et,Extra Trees Regressor,0.0491,0.0625,0.2492,0.9987,0.0224,0.0119,3.1850
lr,Linear Regression,0.0590,0.0652,0.2535,0.9986,0.0264,0.0237,0.2340
ridge,Ridge Regression,0.0590,0.0652,0.2536,0.9986,0.0264,0.0237,0.2100
br,Bayesian Ridge,0.0591,0.0653,0.2536,0.9986,0.0264,0.0237,0.2860
par,Passive Aggressive Regressor,0.0923,0.0693,0.2618,0.9986,0.0500,0.0373,0.2710
gbr,Gradient Boosting Regressor,0.0596,0.0652,0.2536,0.9986,0.0235,0.0191,9.1500
dt,Decision Tree Regressor,0.0679,0.1146,0.3375,0.9976,0.0300,0.0159,0.6220


In [64]:
df = df_min.copy()
df.drop(['Cycle', 'Rainfall'], axis=1, inplace=True)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
target_col = 1
for col_name in df.columns:
    if not (voltage_cols[target_col] in col_name) and ('Voltage' in col_name):
        df.drop(col_name, axis=1, inplace=True)
df.dropna(inplace=True)
pyc_r.setup(df, target=voltage_cols[target_col])
setup_df = pyc_r.pull()
best_model = pyc_r.compare_models()
compare_df = pyc_r.pull()

,Description,Value
0,Session id,8486
1,Target,FT_Voltage2
2,Target type,Regression
3,Original data shape,"(143940, 72)"
4,Transformed data shape,"(143940, 72)"
5,Transformed train set shape,"(100758, 72)"
6,Transformed test set shape,"(43182, 72)"
7,Numeric features,71
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.0514,0.0577,0.2390,0.9988,0.0247,0.0199,0.3240
et,Extra Trees Regressor,0.0417,0.0562,0.2361,0.9988,0.0218,0.0112,4.6260
ridge,Ridge Regression,0.0515,0.0578,0.2393,0.9988,0.0248,0.0199,0.2280
gbr,Gradient Boosting Regressor,0.0527,0.0578,0.2397,0.9988,0.0229,0.0186,9.0890
lightgbm,Light Gradient Boosting Machine,0.0473,0.0570,0.2379,0.9988,0.0220,0.0162,0.4910
rf,Random Forest Regressor,0.0415,0.0559,0.2356,0.9988,0.0214,0.0111,5.5300
br,Bayesian Ridge,0.0515,0.0577,0.2392,0.9988,0.0248,0.0199,0.2910
dt,Decision Tree Regressor,0.0564,0.0950,0.3068,0.9980,0.0278,0.0148,0.6260
huber,Huber Regressor,0.1430,0.1311,0.3595,0.9972,0.0767,0.0762,1.1330
knn,K Neighbors Regressor,0.1200,0.1492,0.3857,0.9968,0.0459,0.0600,0.7950


In [65]:
df = df_min.copy()
df.drop(['Cycle', 'Rainfall'], axis=1, inplace=True)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
target_col = 2
for col_name in df.columns:
    if not (voltage_cols[target_col] in col_name) and ('Voltage' in col_name):
        df.drop(col_name, axis=1, inplace=True)
df.dropna(inplace=True)
pyc_r.setup(df, target=voltage_cols[target_col])
setup_df = pyc_r.pull()
best_model = pyc_r.compare_models()
compare_df = pyc_r.pull()

,Description,Value
0,Session id,1178
1,Target,DN_Voltage1
2,Target type,Regression
3,Original data shape,"(143940, 72)"
4,Transformed data shape,"(143940, 72)"
5,Transformed train set shape,"(100758, 72)"
6,Transformed test set shape,"(43182, 72)"
7,Numeric features,71
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.0558,0.0633,0.2505,0.9988,0.0269,0.0327,0.3290
et,Extra Trees Regressor,0.0428,0.0596,0.2434,0.9988,0.0222,0.0131,5.9340
ridge,Ridge Regression,0.0558,0.0633,0.2506,0.9988,0.0269,0.0327,0.2220
gbr,Gradient Boosting Regressor,0.0554,0.0627,0.2491,0.9988,0.0234,0.0204,9.0180
lightgbm,Light Gradient Boosting Machine,0.0480,0.0610,0.2457,0.9988,0.0226,0.0179,0.4520
rf,Random Forest Regressor,0.0433,0.0609,0.2457,0.9988,0.0221,0.0131,5.7620
br,Bayesian Ridge,0.0559,0.0634,0.2506,0.9988,0.0270,0.0327,0.3550
omp,Orthogonal Matching Pursuit,0.0584,0.0638,0.2513,0.9988,0.0252,0.0192,0.2010
dt,Decision Tree Regressor,0.0593,0.1088,0.3283,0.9979,0.0293,0.0177,0.6360
huber,Huber Regressor,0.1336,0.1292,0.3587,0.9975,0.0595,0.0554,1.3930


In [66]:
df = df_min.copy()
df.drop(['Cycle', 'Rainfall'], axis=1, inplace=True)
voltage_cols = ['FT_Voltage1', 'FT_Voltage2', 'DN_Voltage1', 'DN_Voltage2']
target_col = 3
for col_name in df.columns:
    if not (voltage_cols[target_col] in col_name) and ('Voltage' in col_name):
        df.drop(col_name, axis=1, inplace=True)
df.dropna(inplace=True)
pyc_r.setup(df, target=voltage_cols[target_col])
setup_df = pyc_r.pull()
best_model = pyc_r.compare_models()
compare_df = pyc_r.pull()

,Description,Value
0,Session id,7774
1,Target,DN_Voltage2
2,Target type,Regression
3,Original data shape,"(143940, 72)"
4,Transformed data shape,"(143940, 72)"
5,Transformed train set shape,"(100758, 72)"
6,Transformed test set shape,"(43182, 72)"
7,Numeric features,71
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.0564,0.0663,0.2561,0.9987,0.0267,0.0295,0.3210
et,Extra Trees Regressor,0.0442,0.0632,0.2506,0.9987,0.0233,0.0138,3.0340
ridge,Ridge Regression,0.0565,0.0665,0.2564,0.9987,0.0267,0.0296,0.2210
gbr,Gradient Boosting Regressor,0.0569,0.0662,0.2561,0.9987,0.0243,0.0212,9.0310
lightgbm,Light Gradient Boosting Machine,0.0499,0.0658,0.2552,0.9987,0.0237,0.0187,0.4860
rf,Random Forest Regressor,0.0446,0.0662,0.2564,0.9987,0.0236,0.0138,5.6840
br,Bayesian Ridge,0.0564,0.0664,0.2562,0.9987,0.0266,0.0295,0.3080
omp,Orthogonal Matching Pursuit,0.0572,0.0654,0.2542,0.9987,0.0252,0.0198,0.1990
par,Passive Aggressive Regressor,0.0935,0.0725,0.2680,0.9986,0.0473,0.0426,0.2640
huber,Huber Regressor,0.0892,0.1117,0.3337,0.9978,0.0427,0.0402,1.1650
